<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/report/football_live_cv_total_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Делает отчет по предиктам в тоталах и визуализрует результат<br>
Важные переменные

In [ ]:
remove_list = ['A1relativ', 'A2relativ', 'DA1relativ', 'DA2relativ'] #Ненужные фичи для предикта
selected_model1_version = 6 #3 #6 # Номер модели1
selected_model2_version = 5 #4 #5 # Номер модели2
folded = True #False #True # Модель на фолдах или нет

### Project config

In [ ]:
try:
    import neptune
except:
    !pip install neptune >> None
    import neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

In [ ]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [ ]:
!pip install catboost >> None

### Downloads

In [ ]:
!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/LQQ0gE8WnirciQ
!unrar x ./data.rar ./ >> None
!mv ./FirstTime ./Result2

In [ ]:
#!wget -q -O ./data.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/bM4vbHBIY7wpoQ
#!mkdir -p ./Result2
#!unrar e ./data.rar ./Result2/

In [ ]:
#!wget -q -O ./Result2.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/MwzjeVAGOZwJaw
#!wget -q -O ./Prem.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/oTq97titK56Y2A
#!mkdir -p ./Prem
#!unrar e ./Prem.rar ./Prem/

In [ ]:
#!wget -q -O ./Index.rar https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/rVBc3eq2SG8Ppw
#!mkdir -p ./Index
#!unrar e ./Index.rar ./Index/

### Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

1.5.3
1.22.4


In [ ]:
np.iinfo(np.int8).max, np.iinfo(np.int16).max, np.iinfo(np.int32).max

(127, 32767, 2147483647)

In [ ]:
np.finfo(np.float16).precision, np.finfo(np.float32).precision, np.finfo(np.float64).precision

(3, 6, 15)

In [ ]:
import os, psutil, time
from glob import glob
from neptune.utils import stringify_unsupported
from tqdm import tqdm
import functools
import gc

In [ ]:
from catboost import CatBoost
from scipy.stats import poisson

### Code

#####Functions

In [ ]:
def define_files_list(data_start:str, folder:str, date_end = '') -> list:
    '''
    data_start, data_end
    folder: ./folder/
    '''
    if date_end == '':
        return sorted(
            [file
            for file in glob(folder + '*')
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start)]
            )
    else:
        return sorted(
            [file
            for file in glob(folder + '*')
            if int(file.split('/')[-1].split('.csv')[0]) >= int(date_start) & \
                int(file.split('/')[-1].split('.csv')[0]) <= int(date_end)]
            )

In [ ]:
def create_id_dict(date_start, folder, cols, date_end = '') -> dict:
    '''
    '''
    return pd.concat(map(
        functools.partial(
            pd.read_csv, sep=';', usecols =  cols
                            ),
                    define_files_list(date_start, folder, date_end = date_end)
                    ), ignore_index=True).drop_duplicates(subset = ['Id'], keep = 'last').set_index('Id').to_dict(orient = 'index')

In [ ]:
def transform_dataset(data_df:pd.DataFrame, remain = []):
    new_match_vector = data_df['Id'] != data_df['Id'].shift(1)
    # трансформируем минуты
    data_df['min_norm'] = data_df['Minute'].astype(np.float32) / 50
    print('1. Минуты посчитаны...')
    # трансформируем голы
    data_df[data_df['Score1'].isna() & new_match_vector] = 0
    data_df['Score1_norm'] = data_df['Score1'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score1'] > 3, ['Score1_norm']] = 1.0
    data_df[data_df['Score2'].isna() & new_match_vector] = 0
    data_df['Score2_norm'] = data_df['Score2'].fillna(method = 'ffill').astype(np.float32) / 4
    data_df.loc[data_df['Score2'] > 3, ['Score2_norm']] = 1.0

    data_df['Score_diff'] = data_df['Score1'].astype(np.int16) - data_df['Score2'].astype(np.int16)
    data_df.loc[data_df['Score_diff'] < -4, ['Score_diff']] = -4
    data_df.loc[data_df['Score_diff'] > 4, ['Score_diff']] = 4
    data_df[[f'Score_cat_{n}' for n in range(1, 10)]] = pd.get_dummies(data_df['Score_diff']).values
    data_df['Score_diff'] = data_df['Score_diff'].astype(np.float32) / np.float32(4.0)
    if 'Score1' not in remain:
        data_df = data_df.drop(['Score1', 'Score2'], axis = 1)
    print('2. Голы посчитаны...')
    #трансформируем атаки
    data_df['A1_scaled'] = data_df['A1'].astype(np.float32) / 75
    data_df.loc[data_df['A1'] >= 60, ['A1_scaled']] = (60 + (data_df['A1'] - 60) / 4) / 75
    data_df['A2_scaled'] = data_df['A2'].astype(np.float32) / 75
    data_df.loc[data_df['A2'] >= 60, ['A2_scaled']] = (60 + (data_df['A2'] - 60) / 4) / 75
    # атаки в минуту
    data_df['A1perMIN'] = data_df['A1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A1perMIN'] > 4, ['A1perMIN']] = np.float32(4.0)
    data_df['A2perMIN'] = data_df['A2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['A2perMIN'] > 4, ['A2perMIN']] = np.float32(4.0)
    # динамика атак
    new_match_vector5 = data_df['Id'] != data_df['Id'].shift(5)
    data_df['A1relativ'] = data_df['A1'].astype(np.float32) - data_df['A1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A1relativ']] = np.float32(0.0)
    data_df['A1relativ'] = data_df['A1relativ'].fillna(0)
    data_df.loc[data_df['A1relativ'] > 15, ['A1relativ']] = np.float32(15.)
    data_df['A2relativ'] = data_df['A2'].astype(np.float32) - data_df['A2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['A2relativ']] =  np.float32(0.0)
    data_df['A2relativ'] = data_df['A2relativ'].fillna(0)
    data_df.loc[data_df['A2relativ'] > 15, ['A2relativ']] = np.float32(15.)
    if 'A1' not in remain:
        data_df = data_df.drop(['A1', 'A2'], axis = 1)
    print('3. Атаки посчитаны...')
    # трансформируем опасные атаки
    data_df['DA1_scaled'] = data_df['DA1'].astype(np.float32) / 50
    data_df.loc[data_df['DA1'] >= 40, ['DA1_scaled']] = (80 + (data_df['DA1'] - 40) / 3) / 100
    data_df['DA2_scaled'] = data_df['DA2'].astype(np.float32) / 50
    data_df.loc[data_df['DA2'] >= 40, ['DA2_scaled']] = (80 + (data_df['DA2'] - 40) / 3) / 100
    # опасные атаки в минуту
    data_df['DA1perMIN'] = data_df['DA1'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA1perMIN'] > 3, ['DA1perMIN']] = np.float32(3.0)
    data_df['DA2perMIN'] = data_df['DA2'].astype(np.float32) / data_df['Minute'].astype(np.float32)
    data_df.loc[data_df['DA2perMIN'] > 3, ['DA2perMIN']] = np.float32(3.0)
    # динамика опасных атак
    data_df['DA1relativ'] = data_df['DA1'].astype(np.float32) - data_df['DA1'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA1relativ']] = np.float32(0.0)
    data_df['DA1relativ'] = data_df['DA1relativ'].fillna(0)
    data_df.loc[data_df['DA1relativ'] > 10, ['DA1relativ']] = np.float32(10.)
    data_df['DA2relativ'] = data_df['DA2'].astype(np.float32) - data_df['DA2'].shift(5).astype(np.float32)
    data_df.loc[new_match_vector5, ['DA2relativ']] = np.float32(0.0)
    data_df['DA2relativ'] = data_df['DA2relativ'].fillna(0)
    data_df.loc[data_df['DA2relativ'] > 10, ['DA2relativ']] = np.float32(10.)
    if 'DA1' not in remain:
        data_df = data_df.drop(['DA1', 'DA2'], axis = 1)
    if 'Minute' not in remain:
        data_df = data_df.drop(['Minute'], axis = 1)
    print('4. Опасные атаки посчитаны...')
    # трансформируем владение мячом
    data_df[data_df['Pos1'].isna() & new_match_vector] = 0
    data_df['Pos1_cleaned'] = data_df['Pos1'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos1_cleaned'] < 0.2, ['Pos1_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos1_cleaned'] > 0.8, ['Pos1_cleaned']] = np.float32(0.8)
    data_df[data_df['Pos2'].isna() & new_match_vector] = 0
    data_df['Pos2_cleaned'] = data_df['Pos2'].fillna(method = 'ffill').astype(np.float32) /  np.float32(100.0)
    data_df.loc[data_df['Pos2_cleaned'] < 0.2, ['Pos2_cleaned']] = np.float32(0.2)
    data_df.loc[data_df['Pos2_cleaned'] > 0.8, ['Pos2_cleaned']] = np.float32(0.8)
    if 'Pos1' not in remain:
        data_df = data_df.drop(['Pos1', 'Pos2'], axis = 1)
    print('5. Владение мячом посчитпно...')
    # трансформируем удары
    data_df[data_df['Off1'].isna() & new_match_vector] = 0
    data_df['Off1_norm'] = data_df['Off1'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off1_norm'] > 1.0, ['Off1_norm']] = np.float32(1.0)
    data_df[data_df['Off2'].isna() & new_match_vector] = 0
    data_df['Off2_norm'] = data_df['Off2'].fillna(method = 'ffill').astype(np.float32) / np.float32(10.0)
    data_df.loc[data_df['Off2_norm'] > 1.0, ['Off2_norm']] = np.float32(1.0)
    if 'Off1' not in remain:
        data_df = data_df.drop(['Off1', 'Off2'], axis = 1)
    print('6. Удары посчитаны...')
    # трансформируем удары в створ
    data_df[data_df['On1'].isna() & new_match_vector] = 0
    data_df['On1_norm'] = data_df['On1'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On1_norm'] > 1.0, ['On1_norm']] = np.float32(1.0)
    data_df[data_df['On2'].isna() & new_match_vector] = 0
    data_df['On2_norm'] = data_df['On2'].fillna(method = 'ffill').astype(np.float32) / np.float32(5.0)
    data_df.loc[data_df['On2_norm'] > 1.0, ['On2_norm']] = np.float32(1.0)
    if 'On1' not in remain:
        data_df = data_df.drop(['On1', 'On2'], axis = 1)
    print('7. Удары в створ посчитаны...')
    # трансформируем желтые карточки
    data_df[data_df['YC1'].isna() & new_match_vector] = 0
    data_df['YC1_transformed'] = data_df['YC1'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC1_transformed'] > 1.0, ['YC1_transformed']] = np.float32(1.0)
    data_df[data_df['YC2'].isna() & new_match_vector] = 0
    data_df['YC2_transformed'] = data_df['YC2'].fillna(0).astype(np.float32) / np.float32(2.0)
    data_df.loc[data_df['YC2_transformed'] > 1.0, ['YC2_transformed']] = np.float32(1.0)
    if 'YC1' not in remain:
        data_df = data_df.drop(['YC1', 'YC2'], axis = 1)
    print('8. Жёлтые карточки посчитаны...')
    # трансформируем красные карточки
    data_df[data_df['RC1'].isna() & new_match_vector] = 0
    data_df['RC1_transformed'] = data_df['RC1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC1_transformed'] > 1, ['RC1_transformed']] = np.int8(1)
    data_df[data_df['RC2'].isna() & new_match_vector] = 0
    data_df['RC2_transformed'] = data_df['RC2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['RC2_transformed'] > 1, ['RC2_transformed']] = np.int8(1)
    if 'RC1' not in remain:
        data_df = data_df.drop(['RC1', 'RC2'], axis = 1)
    print('9. Красные карточки посчитаны...')
    # трансформируем замены
    data_df[data_df['Sub1'].isna() & new_match_vector] = 0
    data_df['Sub1_transformed'] = data_df['Sub1'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub1_transformed'] > 1, ['Sub1_transformed']] = np.int8(1)
    data_df[data_df['Sub2'].isna() & new_match_vector] = 0
    data_df['Sub2_transformed'] = data_df['Sub2'].fillna(0).astype(np.int8)
    data_df.loc[data_df['Sub2_transformed'] > 1, ['Sub2_transformed']] = np.int8(1)
    if 'Sub1' not in remain:
        data_df = data_df.drop(['Sub1', 'Sub2'], axis = 1)
    print('10. Замены посчитаны...')
    # трансформируем угловык
    data_df[data_df['Cor1'].isna() & new_match_vector] = 0
    data_df['Cor1_transformed'] = data_df['Cor1'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor1_transformed'] > 1.0, ['Cor1_transformed']] = np.float32(1.0)
    data_df[data_df['Cor2'].isna() & new_match_vector] = 0
    data_df['Cor2_transformed'] = data_df['Cor2'].fillna(0).astype(np.float32) / np.float32(6.0)
    data_df.loc[data_df['Cor2_transformed'] > 1.0, ['Cor2_transformed']] = np.float32(1.0)
    if 'Cor1' not in remain:
        data_df = data_df.drop(['Cor1', 'Cor2'], axis = 1)
    print('11. Угловые посчитаны...')
    # трансформируем линию
    data_df['P1_transformed'] = np.log(data_df['P1'], dtype = np.float32) / 2
    data_df['P2_transformed'] = np.log(data_df['P2'], dtype = np.float32) / 2
    if 'P1' not in remain:
        data_df = data_df.drop(['P1', 'P2'], axis = 1)
    print('12. Линии посчитаны...')
    if 'Pen1' not in remain:
        data_df = data_df.drop(['Pen1', 'Pen2'], axis = 1)
    if 'Active' not in remain:
        data_df = data_df.drop(['Active'], axis = 1)
    return data_df

#### Predefine data

In [ ]:
import datetime
data_types_dict = {
    'Id':np.int32,
    'StatTime':str,
    'Minute':np.int8,
    'Active': np.int8, 'Score1':np.int8, 'Score2':np.int8,
    'A1':np.int16, 'A2':np.int16, 'DA1':np.int16, 'DA2':np.int16, 'Pos1':np.float32, 'Pos2':np.float32,
    'Off1':np.int8, 'Off2':np.int8, 'On1':np.int8, 'On2':np.int8, 'YC1':np.int8, 'YC2':np.int8,
    'RC1':np.int8, 'RC2':np.int8, 'Sub1':np.int8, 'Sub2':np.int8, 'Pen1':np.int8, 'Pen2':np.int8,
    'Cor1':np.int8, 'Cor2':np.int8, 'Period':np.int8,
    'D':np.datetime64,
    'I':np.int32, 'Active.1':np.int8,
    'Time':np.datetime64,
    'Minute.1':np.int8,
    'RawTime':np.datetime64,
    'Score1.1':np.int8, 'Score2.1':np.int8, 'Period.1':np.int8,
    'W1':np.float16, 'WX':np.float16, 'W2':np.float16, 'X1':np.float16, 'X2':np.float16, 'W12':np.float16, 'TotalValue':np.float16,
    'Over':np.float16, 'Under':np.float16, 'Hand1Value':np.float16, 'H1':np.float16, 'H2':np.float16
}

In [ ]:
k_cols = [
    'W1', 'WX', 'W2', 'X1', 'X2', 'W12', 'TotalValue' ,'Over',
    'Under', 'Hand1Value', 'H1', 'H2'
]

In [ ]:
drop_cols = [
    #'StatTime',
    'Comment', 'D', 'I', 'Time', 'Minute.1', 'RawTime',
    'Score1.1', 'Score2.1', 'Period.1', 'Period', 'Periods', 'Serve'
]

#### Load Parameters

In [ ]:
info_cols = ['Id', 'Result1', 'Result2']
prem_cols = ['Id', 'P1', 'PX', 'P2']
res_cols = [cols for cols in data_types_dict if cols not in drop_cols]

info_folder = './Index/'
prem_folder = './Prem/'
data_folder = './Result2/'

In [ ]:
### собрать датасет с даты:
year_start = '2023'
month_start = '01'
day_start = '01'
### до даты
year_end = ''
month_end = ''
day_end = ''
date_start = year_start + month_start + day_start
date_end = year_end + month_end + day_end

#### Create DataFrame

In [ ]:
info_dict = create_id_dict(date_start, info_folder, info_cols)
prem_dict = create_id_dict(date_start, prem_folder, prem_cols)
if len(prem_dict) != len(info_dict):
    print('Нет в info: ', set(prem_dict) - set(info_dict))
    print('Нет в prem: ', set(info_dict) - set(prem_dict))

In [ ]:
#files_list =['./Result2/' + str(id) + '.csv'
#    for id in set(info_dict)
#    if os.path.exists('./Result2/' + str(id) + '.csv')]
files_list = []
for folder in glob('./Result2/*'):
    files_list += glob(folder + '/*')

In [ ]:
time_point1 = time.time()
data_df = pd.concat(map(functools.partial(
    pd.read_csv,
    sep = ';',
    #parse_dates = ['StatTime'],
    usecols = res_cols
                                    ), files_list), ignore_index=True)
time_point2 = time.time()
print(f'create dataframe: {time_point2 - time_point1} sec.')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

data_df[k_cols] = data_df[k_cols].fillna(0).values # Заполняем пропущенные лайв кэфы 1
new_match_vector = data_df['Id'] != data_df['Id'].shift(1) # Отмечаем переходы между матчами в колонуах
data_cols = list(set(res_cols) - set(k_cols)) # Отбирвем колонки для заполнения пропусков метдом ffill
for col in data_cols:
    data_df.loc[data_df[col].isna() & new_match_vector, col] = 0 # Сами переходы с пропусками заполняем 0
data_df = data_df.fillna(method = 'ffill')
################ Записываем файл с временными метками и сбрасываем время матчей
#(data_df['Id'].astype(str) + ':' + data_df['StatTime'].astype(str).str.split(
#        ' ', expand=True)[0]
#        ).drop_duplicates().str.split(':', expand = True).rename(
#                                                columns = {0:'Id', 1:'StatTime'}
#                                                            ).to_csv('./time.csv', index = False)
#data_df = data_df.drop(['StatTime'], axis = 1)
############################################### Присоединяем результаты и кэфы
data_df[['Result1', 'Result2']] = [
    [info_dict[id]['Result1'], info_dict[id]['Result2']]
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]
data_df[['P1', 'PX', 'P2']] = [
    [prem_dict[id]['P1'], prem_dict[id]['PX'], prem_dict[id]['P2']]
    for id in tqdm(data_df['Id'].values, total = len(data_df))
    ]

print('\n')
#---------------
process = psutil.Process(os.getpid())
print(round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

create dataframe: 192.94762516021729 sec.
2.12 GiB


100%|██████████| 1827026/1827026 [00:03<00:00, 470383.92it/s]




2.01 GiB


In [ ]:
data_df.loc[new_match_vector & (data_df['P1'].isna()), ['P1']] = data_df['W1'][new_match_vector & (data_df['P1'].isna())]
data_df.loc[new_match_vector & (data_df['P2'].isna()), ['P2']] = data_df['W2'][new_match_vector & (data_df['P2'].isna())]
data_df.loc[new_match_vector & (data_df['PX'].isna()), ['PX']] = data_df['WX'][new_match_vector & (data_df['PX'].isna())]
data_df['P1'] = data_df['P1'].fillna(method = 'ffill').values
data_df['PX'] = data_df['PX'].fillna(method = 'ffill').values
data_df['P2'] = data_df['P2'].fillna(method = 'ffill').values

In [ ]:
data_df = transform_dataset(data_df, #)
    remain = ['StatTime', 'Minute', 'Score1', 'Score2', 'Result1', 'Result2', 'P1', 'PX', 'P2'])

1. Минуты посчитаны...
2. Голы посчитаны...
3. Атаки посчитаны...
4. Опасные атаки посчитаны...
5. Владение мячом посчитпно...
6. Удары посчитаны...
7. Удары в створ посчитаны...
8. Жёлтые карточки посчитаны...
9. Красные карточки посчитаны...
10. Замены посчитаны...
11. Угловые посчитаны...
12. Линии посчитаны...


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Из-за того что некоторые кэфы были пропущены, и заменены на нули в препроцессинге на их месте появились inf, удаляем
print('P1 -inf & P2 -inf: ', (data_df['P1_transformed'] == -np.inf).sum(), (data_df['P2_transformed'] == -np.inf).sum())

#data_df = data_df[~(data_df['P1_transformed'] == -np.inf)]
#data_df = data_df[~(data_df['P2_transformed'] == -np.inf)]


P1 -inf & P2 -inf:  1487 1487


In [ ]:
#Там где забито болше внутри матча чем в итоге, пока не удаляем
print('Score-Result error 1&2: ', (~(data_df['Result1'] - data_df['Score1']) >= 0).sum(), (~(data_df['Result1'] - data_df['Score1']) >= 0).sum())
#data_df = data_df.loc[(data_df['Result1'] - data_df['Score1']) >= 0]
#data_df = data_df.loc[(data_df['Result2'] - data_df['Score2']) >= 0]


#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

Score-Result error 1&2:  23 23
mem usage:  2.63 GiB


In [ ]:
time_point1 = time.time()
data_df['StatTime'] = pd.to_datetime(data_df['StatTime'], dayfirst=True)
data_df = data_df.sort_values(by = ['StatTime'])
print(f'sort by date: {time_point2 - time_point1} sec.')
#---------------
process = psutil.Process(os.getpid())
print('mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

sort by date: -22.206292867660522 sec.
mem usage:  2.1 GiB


In [ ]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827026 entries, 474727 to 1433479
Data columns (total 64 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Id                int64         
 1   StatTime          datetime64[ns]
 2   Minute            int64         
 3   Score1            int64         
 4   Score2            int64         
 5   Active.1          int64         
 6   W1                float64       
 7   WX                float64       
 8   W2                float64       
 9   X1                float64       
 10  X2                float64       
 11  W12               float64       
 12  TotalValue        float64       
 13  Over              float64       
 14  Under             float64       
 15  Hand1Value        float64       
 16  H1                float64       
 17  H2                float64       
 18  Result1           int64         
 19  Result2           int64         
 20  P1                float64       
 21  PX 

In [ ]:
data_df.describe()

,Id,Minute,Score1,Score2,Active.1,W1,WX,W2,X1,X2,W12,TotalValue,Over,Under,Hand1Value,H1,H2,Result1,Result2,P1,PX,P2,min_norm,Score1_norm,Score2_norm,Score_diff,Score_cat_1,Score_cat_2,Score_cat_3,Score_cat_4,Score_cat_5,Score_cat_6,Score_cat_7,Score_cat_8,Score_cat_9,A1_scaled,A2_scaled,A1perMIN,A2perMIN,A1relativ,A2relativ,DA1_scaled,DA2_scaled,DA1perMIN,DA2perMIN,DA1relativ,DA2relativ,Pos1_cleaned,Pos2_cleaned,Off1_norm,Off2_norm,On1_norm,On2_norm,YC1_transformed,YC2_transformed,RC1_transformed,RC2_transformed,Sub1_transformed,Sub2_transformed,Cor1_transformed,Cor2_transformed,P1_transformed,P2_transformed
count,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06,1.827026e+06
mean,6.082924e+06,2.294666e+01,3.080389e-01,2.443277e-01,8.773542e-01,3.053094e+00,3.690139e+00,4.655567e+00,1.342247e+00,1.799202e+00,1.055450e+00,2.426204e+00,1.713796e+00,1.692445e+00,-2.627609e-01,1.742895e+00,1.749889e+00,1.570168e+00,1.256676e+00,2.810972e+00,4.070412e+00,4.132569e+00,4.589331e-01,7.688218e-02,6.102664e-02,1.586691e-02,7.695566e-04,3.569736e-03,2.061547e-02,1.247273e-01,6.547083e-01,1.571384e-01,3.077570e-02,6.258805e-03,1.436761e-03,3.255832e-01,3.097913e-01,1.095956e+00,1.042964e+00,4.900517e+00,4.660498e+00,2.678083e-01,2.404855e-01,5.799673e-01,5.201102e-01,2.799123e+00,2.509011e+00,4.234815e-01,4.013396e-01,1.442812e-01,1.237681e-01,2.001240e-01,1.688512e-01,1.102658e-01,1.212815e-01,6.084752e-03,6.889338e-03,2.713207e-02,2.836084e-02,1.908505e-01,1.621464e-01,-inf,-inf
std,4.531704e+05,1.296025e+01,6.040536e-01,5.340757e-01,3.280302e-01,4.210603e+00,2.597617e+00,5.946112e+00,1.304684e+00,1.704582e+00,4.740553e-01,1.277265e+00,6.132216e-01,6.018021e-01,1.211181e+00,6.973025e-01,6.999680e-01,1.429285e+00,1.270992e+00,3.094289e+00,1.923771e+00,4.160699e+00,2.592049e-01,1.500676e-01,1.330576e-01,1.948013e-01,2.773022e-02,5.964055e-02,1.420932e-01,3.304095e-01,4.754634e-01,3.639313e-01,1.727095e-01,7.886467e-02,3.787740e-02,2.059745e-01,1.976283e-01,4.328578e-01,4.216070e-01,3.116739e+00,3.008889e+00,1.987007e-01,1.845585e-01,3.209292e-01,3.067699e-01,2.273073e+00,2.151310e+00,1.777128e-01,1.664423e-01,1.640837e-01,1.476538e-01,2.435194e-01,2.228292e-01,2.422440e-01,2.530926e-01,7.776716e-02,8.271565e-02,1.624683e-01,1.660016e-01,2.258420e-01,2.070269e-01,NaN,NaN
min,4.975839e+06,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-02,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-inf,-inf
25%,5.736383e+06,1.200000e+01,0.000000e+00,0.000000e+00,1.000000e+00,1.290000e+00,2.820000e+00,1.570000e+00,1.023000e+00,1.109000e+00,1

In [ ]:
selected_cols = [item for item in data_df.columns[23:] if item not in remove_list]
#selected_cols = [item for item in data_df.columns[23:]]

In [ ]:
X = data_df[selected_cols].values
y_regression1 = ((data_df['Result1'] - data_df['Score1']) / 21).values
y_regression2 = ((data_df['Result2'] - data_df['Score2']) / 21).values
line = data_df[['TotalValue', 'Over', 'Under']].values
active_rows = np.ones(line.shape[0], dtype = bool)
#active = data_df['Active.1'].values == 1
#active_rows = (active & (line.sum(axis = 1) > 3))
scores1 = data_df['Score1'].values
scores2 = data_df['Score2'].values

#### Download models and restore predicts

In [ ]:
!mkdir -p ./models

In [ ]:
# Модель для 1-ой команды
model_num = selected_model1_version # Указываем номер модели
neptune_model = 'FOOT-LIVEBST1'
neptune_model_version = neptune_model + f'-{model_num}'
model_version_params = dict(
    project = 'scomesse/football',
    model = neptune_model,
    api_token = api_key,
    with_id = neptune_model_version
)
if folded:
    model_version = neptune.init_model_version(**model_version_params)
    description_dict = model_version[f'model_reg1_description'].fetch()
    for kfold_num in range(description_dict['kfold_splits']):
        model_version[f'/models/model_reg1_{kfold_num}'].download(f'./models/booster_reg1_{kfold_num}.model')
    model_version.stop()
else:
    PATH_TO_MODEL = './booster.model'
    model_version = neptune.init_model_version(**model_version_params)
    model_version['model'].download(PATH_TO_MODEL)
    params1 = model_version['team_parameters'].fetch()
    model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-6
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-6/metadata


In [ ]:
model_num_2 = selected_model2_version # Указываем номер модели
neptune_model_2 = 'FOOT-LIVEBST2'
neptune_model_version_2 = neptune_model_2 + f'-{model_num_2}'
model_version_params2 = dict(
    project = 'scomesse/football',
    model = neptune_model_2,
    api_token = api_key,
    with_id = neptune_model_version_2
)
if folded:
    model_version2 = neptune.init_model_version(**model_version_params2)
    description_dict2 = model_version2[f'model_reg2_description'].fetch()
    for kfold_num in range(description_dict2['kfold_splits']):
        model_version2[f'/models/model_reg2_{kfold_num}'].download(f'./models/booster_reg2_{kfold_num}.model')
    model_version2.stop()
else:
    PATH_TO_MODEL = './booster2.model'
    model_version2 = neptune.init_model_version(**model_version_params2)
    model_version2['model'].download(PATH_TO_MODEL)
    params2 = model_version2['team_parameters'].fetch()
    model_version2.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST2/v/FOOT-LIVEBST2-5
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST2/v/FOOT-LIVEBST2-5/metadata


In [ ]:
if folded:
    preds_dict = {}
    preds_dict = {1:sum(
        CatBoost().load_model(
            f'./models/booster_reg1_{kfold_num}.model'
                            ).predict(X) * 21.0
        for kfold_num in range(description_dict['kfold_splits'])
                            ) / description_dict['kfold_splits']}

    preds_dict.update({2:sum(
        CatBoost().load_model(
            f'./models/booster_reg2_{kfold_num}.model'
                            ).predict(X) * 21.0
        for kfold_num in range(description_dict['kfold_splits'])
                            ) / description_dict['kfold_splits']})
else:
    preds_dict = {}
    booster = CatBoost()
    booster.load_model('./booster.model')
    preds_dict = {1:booster.predict(X) * 21.0}

    booster2 = CatBoost()
    booster2.load_model('./booster2.model')
    preds_dict.update({2:booster2.predict(X) * 21.0})

#---------------
process = psutil.Process(os.getpid())
print('Mem usage: ', round(process.memory_info().rss / 1024 ** 3, 2), 'GiB')  # in bytes
#---------------

Mem usage:  2.29 GiB


#### Calculate probabilities

In [ ]:
# Считаем Пуассона
poisson_dict = {}
poisson_dict[1] = {}
poisson_dict[2] = {}
for goal in range(7):
    poisson_dict[1][goal] = poisson.pmf(goal, preds_dict[1][active_rows])
    poisson_dict[2][goal] = poisson.pmf(goal, preds_dict[2][active_rows])

In [ ]:
# Считаем вероятности суммы забитых мячей
total_matrix = np.zeros((np.sum(active_rows), 13))
for goal1 in range(7):
    for goal2 in range(7):
        total_matrix[:, goal1 + goal2] = total_matrix[:, goal1 + goal2] + \
        poisson_dict[1][goal1] * poisson_dict[2][goal2]

In [ ]:
# Считаем вероятности забить не менее определенного количества мячей
over_matrix = np.hstack((
    np.flip(np.cumsum(np.flip(total_matrix, axis = 1), axis = 1), axis = 1),
    np.zeros(np.sum(active_rows)).reshape(-1,1)
                                ))[:, 1:]

In [ ]:
# Считаем вероятности забить не более определенного количества
under_matrix = np.hstack((
        np.zeros(np.sum(active_rows)).reshape(-1,1),
        np.cumsum(total_matrix, axis = 1)
                                        ))[:, :-1]

In [ ]:
# Считаем текущей стэйт суммы забитых мячей
total_state = line[:,0][active_rows] - scores1[active_rows] - scores2[active_rows]
total_state[total_state < 0] = 0

In [ ]:
prob = np.hstack((
    np.take_along_axis(over_matrix, total_state.reshape(-1, 1).astype(np.int8), axis = 1),
    np.take_along_axis(under_matrix, (total_state.reshape(-1, 1) + 0.5).astype(np.int8), axis = 1),
))

#### Report

In [ ]:
data_df[['Over_prob', 'Under_prob']] = prob

In [ ]:
data_df[['O_prob_adj', 'U_prob_adj']] = prob / np.sum(prob, axis = 1).reshape(-1,1)

In [ ]:
report_cols = ['Id', 'StatTime', 'Minute', 'Active.1', 'Score1', 'Score2',
               'Result1', 'Result2', 'TotalValue', 'Over', 'Under',
               'Over_prob', 'Under_prob', 'O_prob_adj', 'U_prob_adj']

In [ ]:
data_df[report_cols][pd.to_datetime(data_df['StatTime']) > pd.to_datetime('2023-06-03 00:00:00')]

,Id,StatTime,Minute,Active.1,Score1,Score2,Result1,Result2,TotalValue,Over,Under,Over_prob,Under_prob,O_prob_adj,U_prob_adj
919785,6668450,2023-06-03 00:00:27,27,1,1,0,1,2,3.5,1.71,2.11,0.466670,0.531469,0.467540,0.532460
919786,6668450,2023-06-03 00:01:21,28,1,1,0,1,2,3.5,1.79,2.00,0.467639,0.530228,0.468639,0.531361
919787,6668450,2023-06-03 00:02:19,29,1,1,0,1,2,3.5,1.77,2.03,0.470173,0.527531,0.471255,0.528745
1430269,6724987,2023-06-03 00:02:50,1,1,0,0,0,1,2.5,1.88,1.90,0.482551,0.516506,0.483007,0.516993
919788,6668450,2023-06-03 00:03:08,30,1,1,0,1,2,3.5,1.80,1.99,0.470339,0.527243,0.471479,0.528521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433475,6063258,2023-06-04 00:36:04,41,0,0,0,0,0,0.0,0.00,0.00,0.719620,0.000000,1.000000,0.000000
1433476,6063258,2023-06-04 00:36:39,42,0,0,0,0,0,0.0,0.00,0.00,0.713450,0.000000,1.000000,0.000000
1433477,6063258,2023-06-04 00:37:59,43,0,0,0,0,0,0.0,0.00,0.00,0.711043,0.000000,1.000000,0.000000
1433478,6063258,2023-06-04 00:38:42,44,0,0,0,0,0,0.0,0.00,0.00,0.704158,0.000000,1.000000,0.000000


In [ ]:
data_df[(data_df['StatTime'] > '2023-05-15 23:59:59') &\
                     (data_df['StatTime'] < '2023-05-16 12:00:59')].to_csv('./result_report_total.csv', index = None)

#### Upload to Yandex

In [ ]:
data_df[report_cols].to_csv('./total_probabilities.csv', index = False)

In [ ]:
#data_df.to_csv('./total_dataframe.csv', index = False)

In [ ]:
try:
    import yadisk
except:
    !pip install yadisk >> None
    import yadisk
def get_credential2(frmwork = 'yandex token'):
    with open('cred.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw
_, api_key2 = get_credential2()
y = yadisk.YaDisk(token = api_key2)
print('Token exists: ', y.check_token())

Token exists:  True


In [ ]:
date_upload = 20230604

In [ ]:
#print(y.mkdir(f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}'))

In [ ]:
with open('/content/total_probabilities.csv', 'rb') as f:
    y.upload(f, f'disk:/sport_prediction/calcio_2021/uploads/{date_upload}/total_cv_probabilities_wo_folds.csv')

### visualize results

#### assign data

In [ ]:
results_dict = {'validation':{
    'result1':data_df['Result1'].values,
    'result2':data_df['Result2'].values
    }}
line_dict = {'validation':data_df[['TotalValue', 'Over', 'Under']].values}
active_rows_dict = {'validation':\
                    #((data_df['TotalValue'] - data_df['Score1'] - data_df['Score2']) > 2.0) & \
                    (data_df['Over'] > 1) & \
                    (data_df['Under'] > 1) & \
                    (data_df['Active.1'] > 0)
                    }
prob_dict = {'validation':prob}

#### imports

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#### functions

In [ ]:
def calculate_multiclass(probability_2x:np.array, line_2x:np.array):
    '''
    input : 0 - over, 1 - under
    output: 0 - over, 1 - under
    '''
    probability_adj = probability_2x / np.sum(probability_2x, axis = 1).reshape(-1, 1)
    probline_adj = probability_adj * line_2x
    probline = probability_2x * line_2x
    best_odd_result = np.argmax(probline_adj, axis = 1)
    best_odd_float_adj = np.take_along_axis(probline_adj, best_odd_result.reshape(-1, 1), axis = 1)
    best_odd_float = np.take_along_axis(probline, best_odd_result.reshape(-1, 1), axis = 1)
    return {
        'argmax':best_odd_result,
        'float':best_odd_float,
        'float_adj':best_odd_float_adj
    }

In [ ]:
def make_filter(hda, preds_int):
    hda_dict = {'over':0, 'under':1}
    if hda == 'all':
        return preds_int > -1
    else:
        return preds_int == hda_dict[hda]

In [ ]:
def plot_ga_equity_separate(
    sel_best_dict = {'over':1.19, 'under':1.19}, #{'over':1.1675, 'under':1.1675}, #
    prob_dict = prob_dict,
    line_dict = line_dict,
    results_dict = results_dict,
    active_rows_dict = active_rows_dict,
    x_date = np.empty(()),
    neptune_model_version = neptune_model_version,
    data_split = 'validation'
    ):
    hda_dict = {'all':{'ltype':'solid', 'colors':'rgb(93, 164, 214)', 'line_width':3},
                'over':{'ltype':'dot', 'colors':'rgb(255, 144, 14)', 'line_width':1},
                'under':{'ltype':'dot', 'colors':'rgb(44, 160, 101)', 'line_width':1}}
    layout_dict = {}
    data_list = []
    preds_int, preds_float, preds_float_adj = calculate_multiclass(
        prob_dict[data_split],
        line_dict[data_split][:,1:3]).values()
    preds_float_adj = preds_float_adj[:,0]
    preds_vec = preds_int * (2) - 1
    final_goal_sum = (results_dict[data_split]['result1'] + \
                    results_dict[data_split]['result2'])
    res_vec = ((final_goal_sum > line_dict[data_split][:,0]) * (-1) + \
            (final_goal_sum < line_dict[data_split][:,0]) * (1))
    win_vec, lose_vec = (preds_vec * res_vec) == 1, (preds_vec * res_vec) == -1
    line_vec = np.take_along_axis(
                        line_dict[data_split][:,1:3],
                        preds_int.reshape(-1, 1),
                        axis = 1
                                )[:, 0]
    placed_bets = (
        (preds_float_adj >= sel_best_dict['over']) | \
        (preds_float_adj >= sel_best_dict['under'])
                    ) & \
        active_rows_dict[data_split]
    ################
    line_vec = line_vec[placed_bets]
    win_vec = win_vec[placed_bets]
    lose_vec = lose_vec[placed_bets]
    preds_int = preds_int[placed_bets]
    x_date = pd.to_datetime(x_date).dt.date.values[placed_bets]
    ################
    y = ((line_vec - 1) / line_vec) * (win_vec).astype(np.float32) - \
        (1/line_vec) * (lose_vec).astype(np.float32)
    side_dict = {'over':0, 'under':1}
    for hda, value_dict in hda_dict.items():
        if hda != 'all':
            data_list += [
                go.Scatter(
                    y = np.cumsum(y * (preds_int == side_dict[hda])),
                    x = x_date,
                    name = hda,
                    line = dict(color = value_dict['colors'], width = value_dict['line_width'], dash = value_dict['ltype'])
                            )]
        else:
            data_list += [
                go.Scatter(
                    y = np.cumsum(y),
                    x = x_date,
                    name = 'all',
                    line = dict(color = hda_dict['all']['colors'], width = hda_dict['all']['line_width'], dash = hda_dict['all']['ltype'])
                            )]
    title_text = f'<b>Total(cv)</b> equity curve {data_split}<br>Model in neptune.ai: FOOT-{neptune_model_version}<br>' + \
        f"ROI: {np.round(np.cumsum(y)[-1] * 100 / np.sum(1/line_vec, dtype = np.float32), 4)}%" + \
        f' | Bet quantity: {np.sum(placed_bets)}<br>'
    dt_min = min(x_date)
    dt_max = max(x_date)
    layout_dict.update({
    'width':1200,
    'height':800,
    #'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    'xaxis':{'range':[dt_min, dt_max]}
                    })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout)

### visualization

In [ ]:
#active_rows_dict['validation'] = ((data_df['Active.1'].values == 1) & (np.sum(line_dict['validation'][:, 1:3], axis = 1) > 2))
224706
2.631

2.631

In [ ]:
2.947
213261

213261

In [ ]:
fig = plot_ga_equity_separate(
    x_date = data_df['StatTime'],
    #sel_best_dict = {'over':1.19, 'under':1.19}
    )
fig.show()

### Betting Strategies

##### Functions

In [ ]:
def bet_open(match_id_dict:dict, match_minute:int, strategy = '5minute', nbets = 3, last_type = None, current_prob = None, prob_thres = 1.05):
    '''
    '''
    if strategy == '5minute':
        if 'last_bet_minute' in match_id_dict:
            if (match_minute - match_id_dict['last_bet_minute']) > 4:
                return True
            else:
                return False
        else:
            return True
    elif strategy == 'nbets':
        if 'nbets' in match_id_dict:
            if match_id_dict['nbets'] >= nbets:
                return False
            else:
                return True
        else:
            return True
    elif strategy == 'changes':
        if 'last_type' in match_id_dict:
            if match_id_dict['last_type'] != last_type:
                return True
            elif (current_prob / match_id_dict['last_prob']) > prob_thres:
                return True
            else:
                return False
        else:
            return True

In [ ]:
def plot_strategy(
    prob_dict,
    line_dict,
    results_dict,
    active_rows_dict,
    data_df,
    strategy_type = 'changes', #'nbets' #'5minute'
    neptune_model_version = neptune_model_version,
    data_split = 'validation',
    sel_best_dict = {'over':1.19, 'under':1.19}
):

    type2int = {0:'over', 1:'under'}
    hda_dict = {'all':{'ltype':'solid', 'colors':'rgb(93, 164, 214)', 'line_width':3, 'order':2},
                'over':{'ltype':'dot', 'colors':'rgb(255, 144, 14)', 'line_width':1, 'order':0},
                'under':{'ltype':'dot', 'colors':'rgb(44, 160, 101)', 'line_width':1, 'order':1}}

    match_live_dict = {}
    output_list = []
    date_list = []
    bet_size_list = []
    preds_int, preds_float, preds_float_adj = calculate_multiclass(
                                        prob_dict[data_split],
                                        line_dict[data_split][:,1:3]
                                    ).values()
    for pred_win, pred_prob, line, results1, results2, match_id, \
                    match_time, match_minute, active in tqdm(zip(
            preds_int,
            preds_float_adj,
            line_dict[data_split],
            results_dict[data_split]['result1'],
            results_dict[data_split]['result2'],
            data_df['Id'].values,
            data_df['StatTime'],
            data_df['Minute'].values,
            active_rows_dict[data_split]
                                ), total = active_rows_dict[data_split].shape[0]):

        if active:
            if pred_prob > sel_best_dict[type2int[pred_win]]:
                if match_id not in match_live_dict:
                    match_live_dict[match_id] = {'nbets':0}

                if bet_open(match_live_dict[match_id], match_minute, strategy = strategy_type,
                            nbets = 3,
                            last_type = pred_win, current_prob = pred_prob, prob_thres = 1.01):
                    bet_size = 1 / line[pred_win + 1]
                    final_goal_sum = results1 + results2

                    if final_goal_sum == line[0]:
                        event_list = [0, 0]
                        #event_list[pred_win] =  0
                        output_list.append(event_list)
                        date_list.append(match_time.date())
                        bet_size_list.append(bet_size)
                        match_live_dict[match_id]['last_bet_minute'] = match_minute
                        match_live_dict[match_id]['nbets'] += 1
                        match_live_dict[match_id]['last_type'] = pred_win
                        match_live_dict[match_id]['last_prob'] = pred_prob
                    elif pred_win.astype(bool) == (final_goal_sum < line[0]):
                        event_list = [0, 0]
                        event_list[pred_win] =  bet_size * (line[pred_win + 1] - 1)
                        output_list.append(event_list)
                        date_list.append(match_time.date())
                        bet_size_list.append(bet_size)
                        match_live_dict[match_id]['last_bet_minute'] = match_minute
                        match_live_dict[match_id]['nbets'] += 1
                        match_live_dict[match_id]['last_type'] = pred_win
                        match_live_dict[match_id]['last_prob'] = pred_prob
                    else:
                        event_list = [0, 0]
                        event_list[pred_win] = -bet_size
                        output_list.append(event_list)
                        date_list.append(match_time.date())
                        bet_size_list.append(bet_size)
                        match_live_dict[match_id]['last_bet_minute'] = match_minute
                        match_live_dict[match_id]['nbets'] += 1
                        match_live_dict[match_id]['last_type'] = pred_win
                        match_live_dict[match_id]['last_prob'] = pred_prob

    layout_dict = {}
    data_list = []
    for hda, value_dict in hda_dict.items():
        if hda != 'all':
            data_list += [
                go.Scatter(
                    x = date_list,
                    y = np.cumsum(np.array(output_list)[:, value_dict['order']]),
                    name = hda,
                    line = dict(color = value_dict['colors'], width = value_dict['line_width'], dash = value_dict['ltype'])
                            )]
    cumsum_all = np.cumsum(np.sum(np.array(output_list), axis = 1))
    data_list += [
        go.Scatter(
            x = date_list,
            y = cumsum_all,
            name = 'all',
            line = dict(color = hda_dict['all']['colors'], width = hda_dict['all']['line_width'], dash = hda_dict['all']['ltype'])
                    )]

    title_text = f'<b>Total(cv)</b>  equity curve for strategie <b><i>{strategy_type}</i></b>' + \
        f'<br>Model in neptune.ai: FOOT-{neptune_model_version}<br>' + \
        f"ROI: {round(100 * cumsum_all[-1] / sum(bet_size_list), 2)}%" + \
        f' | Bet quantity: {len(output_list)}<br>'
    #dt_min = min(x_date[new_ard])
    #dt_max = max(x_date[new_ard])
    layout_dict.update({
    'width':1200,
    'height':800,
    #'title_x':0.5,
    'title_text':title_text,
    'paper_bgcolor':'rgb(229, 237, 247)',
    'plot_bgcolor':'rgb(229, 237, 247)',
    #'xaxis':{'range':[dt_min, dt_max]}
                    })
    layout = go.Layout(**layout_dict)
    return go.Figure(data=data_list, layout=layout) #, match_live_dict

##### Plot

In [ ]:
fig = plot_strategy(
    prob_dict,
    line_dict,
    results_dict,
    active_rows_dict,
    data_df,
    strategy_type = 'changes', #'changes', #'nbets' #'5minute'
    #neptune_model_version = neptune_model_version,
    #data_split = 'validation',
    #sel_best_dict = {'home':1.1465, 'draw':1.1465, 'away':1.1465}
)
fig.show()